In [76]:
from pathlib import Path
from IPython.display import HTML, display
import ipywidgets as widgets
import datetime
import pandas as pd
import numpy as np
#print(Path.cwd().parent.parent.parent)
base_path = Path.cwd().parent.parent.parent / "runs"
#img_path = Path.cwd() / "img.jpg"
# display(Image(filename=img_path))
p = Path(base_path)
#p.as_uri()

In [77]:
tsts = ["k8s_taewa_3lite_","asg_taewa_3lite_", "k8s_apache_3_", "k8s_taewa_3_", "asg_apache_3_", "asg_taewa_3_"]
tsts = tsts + ["asg_taewa_3extra_", "k8s_taewa_3extra_", "asg_raupi_3_", "k8s_raupi_3_"]
test = widgets.Dropdown(
       options=tsts,
       value='k8s_apache_3_',
       description='Test:')
all = widgets.Checkbox(value=True,
       description='All')
graphs=["QPS_estimatedProcessedBytes.png","QPS_cpuUtilization.png","QPS_groupInServiceCapacity.png"]
date_filter = widgets.Text(
       value='2022.0[3-9]', # use glob pattern http://pymotw.com/2/glob/
       description='Filter', )
box = widgets.HBox([date_filter, all, test ])
display(box)

In [78]:
#print(test._options_values)
tests=[]
if all.value:
    tests=test._options_values;
else:
    tests.append(test.value)

for tst in tests:
    search=f'*{date_filter.value}*{tst}*'
    nw = datetime.datetime.now()
    count=0
    iList=[]
    avg_qps_list=[]
    avg_cpu_list=[]
    avg_duration_list=[]
    total_err_list=[]

    for i in p.glob(search):
        print(i)
        count+=1
        dir = i.name
        if not (i/"csv").exists():
            line = f"<tr><td rowspan='1'><b>skip skip skip: {dir}</b></td></tr>"
            iList.append(line)
            continue
        ##================ get json Fortio data ================
        #print(i)
        json_paths = (i/"csv").glob('*.json')
        fortio_path=list(json_paths)[0]

        df = pd.read_json(fortio_path, lines=True)
        df['StartTime'] = pd.to_datetime(df['StartTime'])
        df.set_index(['StartTime'],inplace=True)

        #qps= df[["ActualQPS","NumThreads"]]
        #mydf = df[df['Labels'] == 'k8s_apache_3-performance-1']

        # get performance QPS
        perf_df = df[df['Labels'].str.contains('performance-[1-3]', regex=True)]
        #perf_data=perf_df.ActualQPS
        #qps_lst=list(map(int,perf_data))
        qps_lst=list(perf_df.ActualQPS)
        avg_qps=np.mean(qps_lst)
        avg_qps_list.append(avg_qps)

        # get scaling start times
        scaling_starts_df = df[df['Labels'].str.contains('scaling-.-1$', regex=True)]
        #print(df)
        #print(scaling_df['Labels'])

        aws_metric0="backendConnectionErrors"
        c_path_err=list((i/"csv").glob('*'+aws_metric0+'.csv'))[0]
        err_df = pd.read_csv(c_path_err, parse_dates=['datetime'], index_col="datetime")

        ## ================ PERFORMANCE CloudWatch data ================
        aws_metric1="cpuUtilization"
        csv_paths = (i/"csv").glob('*'+aws_metric1+'.csv')
        c_path=list(csv_paths)[0]
        mdf = pd.read_csv(c_path, parse_dates=['datetime'], index_col="datetime")
        perfomance_cpu_list=[]
        #print(c_path)
        for index,value in perf_df.iterrows():
            perf_start=index#pd.to_datetime(stime)
            #print(perf_start)
            
            # get 4 max:
            # mmdf=mdf[perf_start:perf_start+ pd.Timedelta(5, "m")]
            # mmmdf=mmdf.nlargest(4, aws_metric1).sort_index()
            # get 3 mid:
            
            mmmdf=mdf[perf_start+ pd.Timedelta(1, "m"):perf_start + pd.Timedelta(4, "m")]
            cpu_list=list(mmmdf[aws_metric1])
            average_cpu=np.mean(cpu_list)
            perfomance_cpu_list.append(average_cpu)

            #print(list(mmmdf.cpuUtilization))
            if value.Labels[-1] == '1':
                first_perf_start=index
                #print(value.Labels)

        # get number of errors from the start of first performance till the end of last scaling
        merr_df=err_df[first_perf_start:first_perf_start + pd.Timedelta(68, "m")] 
        err_list=list(merr_df[aws_metric0])
        total_err_list.append(sum(err_list))
        #print(total_err_list)

        avg_cpu=np.mean(perfomance_cpu_list)
        avg_cpu_list.append(avg_cpu)

        ## ================ SCALING CloudWatch data ================
        aws_metric2="groupInServiceCapacity"
        scaling_durations=[]
        csv_paths2 = (i/"csv").glob('*'+aws_metric2+'.csv')
        c_path2=list(csv_paths2)[0]
        cwdf2 = pd.read_csv(c_path2, parse_dates=['datetime'], index_col="datetime")
        #print(c_path2)
        for index,value in scaling_starts_df.iterrows():
            scale_start=index
            #print(scale_start)
            
            ccwdf2=cwdf2[scale_start:scale_start+ pd.Timedelta(14, "m")] # only need to look 14 min ahead as it's the length of the scaling run
            max_scaled=ccwdf2[ccwdf2[aws_metric2].ge(3)] #.index[0]

            # print(ccwdf2.head(6))
            # print(max_scaled.head(2))

            if max_scaled.size > 0:
                cw_reach_max_time=max_scaled.index[0]
                duration = cw_reach_max_time - index
                duration_in_s = duration.total_seconds()
                #minutes = divmod(duration_in_s, 60)[0]
                #print(reach_max_time)
                #s = scaling_starts_df.index.get_loc(cw_reach_max_time, method='nearest')
                #max_minute=scaling_starts_df[reach_max_time]
                
                # print(index)
                # print(cw_reach_max_time)
                scaling_durations.append(duration_in_s)
            else:
                duration_in_s = 0
        avg_duration=np.mean(scaling_durations)
        avg_duration_list.append(avg_duration)

        ##================ generate rows ================
        line = ""
        line = f"""<tr><td rowspan='1'>{count}:  <a href='{i.name}'>{dir}</a> </td></tr><tr>
                <td>column1</td><td>column2</td><td>column3</td>
                <td>Dura-tion s</td><td>QPS</td><td>CPU %</td>
                <td>Err</td>
                </tr><tr>"""
        ####====== Images
        for y in graphs:
            #print(i.name)
            img=f'{i.name}/csv/{y}'
            line+=f"<td><img alt='{y}' src='{img}' /></td>"

        ####====== Data

        #data=str(lst)[1:-1].replace(", ", "<br>")
        line+=f"<td>{list(map(int,scaling_durations))}<br><b>{avg_duration:.0f}</b></td>"
        line+=f"<td>{list(map(int,qps_lst))}<br><b>{avg_qps:.0f}</td>"
        line+=f"<td>{list(map(int,perfomance_cpu_list))}<br><b>{avg_cpu:.0f}</b></td>"
        line+=f"<td><b>{total_err_list[-1]}</b></td>"

        line+="</tr>"
        #line+="</tr><tr><td rowspan='1'>~~~</td></tr>"
        iList.append(line)
    
    total=f"""<td></td><td></td><td></td>
        <td>max:<br>{max(avg_duration_list):.0f}<br>min:<br>{min(avg_duration_list):.0f}<br>mean:<br>{np.mean(avg_duration_list):.0f}<br></td>
        <td>max:<br>{max(avg_qps_list):.0f}<br>min:<br>{min(avg_qps_list):.0f}<br>mean:<br>{np.mean(avg_qps_list):.0f}<br></td>
        <td>max:<br>{max(avg_cpu_list):.0f}<br>min:<br>{min(avg_cpu_list):.0f}<br>mean:<br>{np.mean(avg_cpu_list):.0f}<br></td>
        <td>max:<br>{max(total_err_list):.0f}<br>min:<br>{min(total_err_list):.0f}<br>mean:<br>{np.mean(total_err_list):.0f}<br></td>"""
    iList.append(total)
    iList.insert(0,total)
    imagesList = ''.join(iList)

    ##================ save HTML ================

    html_path=p/f'o_{tst}.html'
    header=f"<head><title>{tst}</title>"
    header+="<style>table, th, td {border: 1px solid black;font-size: 20px;} body {font: 20px Arial, sans-serif;}</style></head>"
    header+=f"date: {nw}<br><h2>{tst} ({count})</h2><br>"

    ##================ generate menu
    def sorter(item):
        """sort by app and nodes"""
        x = ''.join(item.name.split("_")[2:4])
        return x
    header+="<table><tr>"
    files = sorted(p.glob('*.html'), key=sorter)
    for h in files:
        test_name=h.name
        if h.name.find(f'{tst}')>0:
            test_name=f"<b>{test_name}</b>"
        if (files.index(h)% 2) == 0:
            header+=f"<td><a href='{test_name}'>{test_name}</a>&nbsp;"
        else:
            header+=f"<br><a href='{test_name}'>{test_name}</a>&nbsp;</td>"
        
    header+="</tr></table>"
    header+="<table>"
    footer="</table>"

    page= header + imagesList
    a = HTML(page)
    html_src = a.data
    with open(html_path, 'w') as f:
        f.write(html_src)

    csv=""
    csv+=f'{tst},{count},{tst.split("_")[0]},{tst.split("_")[1]},'
    csv+=f'"{list(map(int,avg_duration_list))}",{max(avg_duration_list):.0f},{np.mean(avg_duration_list):.0f},{min(avg_duration_list):.0f},'
    csv+=f'"{list(map(int,avg_qps_list))}",{max(avg_qps_list):.0f},{np.mean(avg_qps_list):.0f},{min(avg_qps_list):.0f},'
    csv+=f'"{list(map(int,avg_cpu_list))}",{max(avg_cpu_list):.0f},{np.mean(avg_cpu_list):.0f},{min(avg_cpu_list):.0f},'
    csv+=f'"{total_err_list}",{max(total_err_list):.0f},{np.mean(total_err_list):.0f},{min(total_err_list):.0f}\n'

    csv_path=p/f'o_summary.csv'
    with open(csv_path, "a") as csvfile:
        csvfile.write(csv)

    display(a, metadata=dict(isolated=True))

c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.26_08-14_k8s_taewa_3lite_9d2d
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.27_09-43_k8s_taewa_3lite_0c47
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.31_23-04_k8s_taewa_3lite_41cf
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.04.01_08-53_k8s_taewa_3lite_41cf
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.04.01_21-08_k8s_taewa_3lite_41cf
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.04.02_06-11_k8s_taewa_3lite_41cf
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.04.03_09-53_k8s_taewa_3lite_41cf
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.04.04_03-52_k8s_taewa_3lite_41cf
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.04.05_08-07_k8s_taewa_3lite_41cf
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.04.05_19-47_k8s_taewa_3lite_41cf


c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.26_10-35_asg_taewa_3lite_865b
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.27_19-31_asg_taewa_3lite_0c47
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.31_09-23_asg_taewa_3lite_41cf
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.31_19-33_asg_taewa_3lite_41cf
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.04.02_04-11_asg_taewa_3lite_41cf
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.04.02_08-33_asg_taewa_3lite_41cf
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.04.03_07-21_asg_taewa_3lite_41cf
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.04.04_06-12_asg_taewa_3lite_41cf
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.04.04_08-54_asg_taewa_3lite_41cf
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.04.05_05-15_asg_taewa_3lite_41cf


c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.15_23-46_k8s_apache_3_4b98
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.16_20-42_k8s_apache_3_4b98
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.22_19-33_k8s_apache_3_54ef
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.23_03-27_k8s_apache_3_54ef
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.24_01-00_k8s_apache_3_54ef
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.25_00-34_k8s_apache_3_54ef
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.25_22-59_k8s_apache_3_54ef
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.28_02-52_k8s_apache_3_0c47
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.29_23-13_k8s_apache_3_41cf
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.31_02-22_k8s_apache_3_41cf
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.04.05_22-42_k8s_apache_3_41cf
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.04.16_19-05_k8s_apache

c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.16_06-52_k8s_taewa_3_4b98
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.17_09-14_k8s_taewa_3_af09
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.21_08-15_k8s_taewa_3_54ef
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.22_08-41_k8s_taewa_3_54ef
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.25_08-50_k8s_taewa_3_54ef
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.26_01-22_k8s_taewa_3_54ef
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.27_23-34_k8s_taewa_3_0c47
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.28_06-43_k8s_taewa_3_0c47
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.30_03-56_k8s_taewa_3_41cf
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.04.04_22-12_k8s_taewa_3_41cf
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.04.06_09-07_k8s_taewa_3_41cf
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.04.17_08-20_k8s_taewa_3_41cf


c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.16_03-17_asg_apache_3_4b98
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.17_00-15_asg_apache_3_4b98
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.22_23-32_asg_apache_3_54ef
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.23_19-06_asg_apache_3_54ef
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.24_03-52_asg_apache_3_54ef
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.24_20-04_asg_apache_3_54ef
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.27_07-06_asg_apache_3_0c47
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.28_19-57_asg_apache_3_41cf
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.30_20-36_asg_apache_3_41cf
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.04.04_19-18_asg_apache_3_41cf
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.04.15_22-03_asg_apache_3_41cf
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.04.16_01-02_asg_apache

c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.15_09-26_asg_taewa_3_4b98
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.16_10-07_asg_taewa_3_4b98
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.21_05-42_asg_taewa_3_54ef
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.23_07-15_asg_taewa_3_54ef
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.24_08-16_asg_taewa_3_54ef
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.24_22-34_asg_taewa_3_54ef
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.25_20-08_asg_taewa_3_54ef
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.29_08-43_asg_taewa_3_41cf
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.30_23-07_asg_taewa_3_41cf
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.04.05_01-03_asg_taewa_3_41cf
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.04.17_04-55_asg_taewa_3_41cf
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.04.20_20-56_asg_taewa_3_41cf


c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.27_04-23_asg_taewa_3extra_0c47
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.04.03_23-20_asg_taewa_3extra_41cf
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.04.07_07-40_asg_taewa_3extra_41cf
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.04.22_08-38_asg_taewa_3extra_41cf
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.04.22_19-56_asg_taewa_3extra_41cf
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.04.22_21-58_asg_taewa_3extra_41cf


c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.27_02-05_k8s_taewa_3extra_0c47
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.04.03_19-10_k8s_taewa_3extra_41cf
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.04.07_02-11_k8s_taewa_3extra_41cf
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.04.07_19-42_k8s_taewa_3extra_41cf
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.04.22_10-59_k8s_taewa_3extra_41cf
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.04.23_00-07_k8s_taewa_3extra_41cf


c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.18_02-19_asg_raupi_3_f03c
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.04.18_20-43_asg_raupi_3_41cf
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.04.19_19-40_asg_raupi_3_41cf
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.04.21_03-19_asg_raupi_3_41cf
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.04.22_00-42_asg_raupi_3_41cf


c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.03.18_04-39_k8s_raupi_3_f03c
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.04.18_05-48_k8s_raupi_3_41cf
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.04.19_03-12_k8s_raupi_3_41cf
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.04.21_00-47_k8s_raupi_3_41cf
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.04.21_21-12_k8s_raupi_3_41cf
c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.04.21_22-13_k8s_raupi_3_41cf
